In [1]:
import os
from ppo import visualize
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.interpolate import spline
from scipy.signal import medfilt

# repaint acc_reward in tanh experiment
# env_name = ["multiposrealright_normal_a_squaresafeinit_staticobs_vmaxdismax_avgmode_timecost_highfre-v0",
#             "multiposrealright_normal_v_withvprev_squaresafeinit_staticobs_vmaxdismax_avgmode_timecost_highfre-v0",
#             "multimodrealright_normal_a_squaresafeinit_staticobs_vmaxdismax_avgmode_timecost_highfre-v0",
#             "multimodrealright_normal_v_withvprev_squaresafeinit_staticobs_vmaxdismax_avgmode_timecost_highfre-v0"]

# log_dir = "./train_models/gym/tanh0903/" 
# test_env="/multiposrealright_normal_v_withvprev_squaresafeinit_staticobs_vmaxdismax_avgmode_timecost_norewvpref_highfre-v0"


# log_dir = "./train_models/gym/worker_step_lidar"
# env_name=["w4s32",
#          "w4s64",
#          "w4s128",
#          "w8s32",
#          "w8s64",
#          "w8s128",
#          "w16s32",
#          "w16s64"]
# test_env="/multi_lidar_v_default_norewvpref-v0"

# log_dir = "./train_models/gym/20181101/primary"
# env_name=["fc512"] #,"fc128"]
# test_env=["/multi_mini_lidar_v_smallerdensersensing_highwvpenalty_norewvpref_training-v1"]

# log_dir = "./train_models/gym/20181107/primary/"
# env_name=["with_rnn","without_rnn"]
# test_env=["/multi_lidar_v_default_norewvpref-v0",
#           "/multi_pos_v_default_norewvpref-v0"]

# log_dir = "./train_models/gym/20190116/primary/indppo/"
# env_name=["seed1"] #,"fc128"]
# test_env=["/multi_pos_v_default_norewvpref-v0"]
# folderdir = []
# for env in env_name:
#     for test in test_env:
#         envdir =os.path.join(log_dir, env+test) 
#         folderdir.append(envdir)

# visualization of reward partition       
# ----auxilary choice----
logs_dir = "./ppo/logs"
date = "190126"
env = "lidar_a-v0" # "pos_v-v0"
method = "clde_unordered_fc_seperate_rnn"
# -----------------------
log_dir = "{}/{}/{}/{}".format(logs_dir,date,env,method)
seed_name=["seed1"] #,"fc128"]

folderdir = []
for seed in seed_name:
    envdir =os.path.join(log_dir, seed, 'log') 
    folderdir.append(envdir)

In [2]:
# color stuff
#colors = ["salmon pink", "tiffany blue", "light lavender", "peach", "light blue grey", "pastel yellow", "foam green",  ]
colors = ["shamrock green","neon red","cerulean","pinkish purple","mango","mustard yellow", "peach", "light blue grey", "pastel yellow"]
sns.set_palette(sns.xkcd_palette(colors))
sns.palplot(sns.xkcd_palette(colors)) #"leather" "pinkish purple"

In [3]:
from scipy.signal import savgol_filter

# convergence
convergence_entry = ['Test index', 'Setting', 'Number of frames', 'Accumulated total rewards'] #Setting
convergence_title = {e:[] for e in convergence_entry}
convergence_data = pd.DataFrame(convergence_title)
bin_size = 50000 # 100
# num_iteration = int(10000000/bin_size) # maybe dummy

# setting = ["P+A","P+V","L+A","L+V"]

# setting = ["w4s32","w4s64","w4s128","w8s32","w8s64","w8s128","w16s32","w16s64"]

setting = seed_name

# setting = ["lidar+v+rnn","pos+v+rnn","lidar+v_nornn","pos+v_nornn"]

index = 0
rew_partition = ["reward","rew_goal","rew_pref","rew_formation","rew_vpenalty"]
#for seed in seed_name:
for partition in rew_partition:
    list_iteration = []
    list_reward = []
    # for test in test_env:
    for seed in seed_name:
        # print (env,test)
        print (seed, partition, folderdir)
        # tx, ty = visualize.load_data(folderdir[index], 1, bin_size) 
        tx, ty = visualize.load_data(folderdir[0], 1, bin_size, rew_partition_record=partition)  
        list_iteration.extend(tx) 
        list_reward.extend(ty)

        # index --> seed
        list_test_index = [index for _ in range(len(list_iteration))]
        # list_setting = [setting[index] for _ in range(len(list_iteration))]
        list_setting = [partition for _ in range(len(list_iteration))]
        frame = [list_test_index, list_setting, list_iteration, list_reward]
        frame = list(map(list, zip(*frame)))

        df = pd.DataFrame(frame, columns=convergence_entry)
        convergence_data = convergence_data.append(df, ignore_index=True)

    index += 1

#xnew = np.linspace(list_iteration.min(),list_iteration.max(),300) #300 represents number of points to make between T.min and T.max

seed1 reward ['./ppo/logs/190126/lidar_a-v0/clde_unordered_fc_seperate_rnn/seed1/log']
seed1 rew_goal ['./ppo/logs/190126/lidar_a-v0/clde_unordered_fc_seperate_rnn/seed1/log']
seed1 rew_pref ['./ppo/logs/190126/lidar_a-v0/clde_unordered_fc_seperate_rnn/seed1/log']
seed1 rew_formation ['./ppo/logs/190126/lidar_a-v0/clde_unordered_fc_seperate_rnn/seed1/log']
seed1 rew_vpenalty ['./ppo/logs/190126/lidar_a-v0/clde_unordered_fc_seperate_rnn/seed1/log']


In [4]:
import os
# fig_dir = '{}/{}'.format("./eval_results/sample_figures", "test_single")
fig_dir = '{}/{}/{}'.format(log_dir, seed_name[0], "repaint") # e.g., ./ppo/logs/190125/polar_pos_v-v0/clde_unordered_fc_seperate_rnn/seed1/repaint
if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)

In [5]:
convergence_entry_prefout = ['Test index', 'Setting', 'Number of frames', 'Accumulated total rewards'] #Setting
convergence_title_prefout = {e:[] for e in convergence_entry_prefout}
convergence_data_prefout = pd.DataFrame(convergence_title_prefout)
for setting in ("reward","rew_goal","rew_formation","rew_vpenalty"):
    convergence_data_prefout = convergence_data_prefout.append(convergence_data[convergence_data['Setting']==setting])

In [6]:
convergence_entry = ['Test index', 'Setting', 'Number of frames', 'Accumulated total rewards'] #Setting
convergence_title = {e:[] for e in convergence_entry}
convergence_data_prefout_rewout = pd.DataFrame(convergence_title)
for setting in ("rew_goal","rew_formation","rew_vpenalty"):
    convergence_data_prefout_rewout = convergence_data_prefout_rewout.append(convergence_data[convergence_data['Setting']==setting])

In [7]:
plt.close()
dashline=[(),(1, 3)] #[(),(2, 2)]
plt.figure(1,figsize = (8,8))
plt.rcParams.update({'font.size': 20}) 
ax = plt.gca()
ax.set_xlabel('Number of frames',fontsize=20)
ax.set_ylabel('Accumulated total rewards',fontsize=20,labelpad =-12)
sns.set_context("paper")
sns.lineplot(x="Number of frames", y="Accumulated total rewards", 
             data=convergence_data_prefout_rewout, ci=None, 
             hue='Setting', dashes=dashline) #, markers=True
legend=ax.legend(loc='lower right',title=None,fontsize=13)
legend.get_title().set_fontsize(fontsize = 8)
plt.tick_params(labelsize=20)
#ax.legend(loc='lower right',title=None)
#plt.legend(loc='lower right',title=None)
# plt.savefig('{}/{}'.format(fig_dir, test_env[0]+env_name[0]+'convergence.png'))
plt.savefig('{}/{}'.format(fig_dir, 'rew_partition_prefout_rewout.png'))

In [8]:
plt.figure(2,figsize = (8,8))
plt.rcParams.update({'font.size': 20}) 
ax = plt.gca()
ax.set_xlabel('Number of frames',fontsize=20)
ax.set_ylabel('Accumulated total rewards',fontsize=20,labelpad =-12)
ax = plt.gca()
sns.set_context("paper")
sns.lineplot(x="Number of frames", y="Accumulated total rewards", 
             data=convergence_data_prefout_rewout, ci=None, 
             hue='Setting', dashes=dashline) #, markers=True
ax.legend(loc='lower right',title=None,fontsize=13)
plt.tick_params(labelsize=20)
plt.savefig('{}/{}'.format(fig_dir, 'rew_partition_prefout_rewout.png'))
plt.show()